In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import warnings
from tqdm import tqdm 
import time
warnings.filterwarnings('ignore')

In [2]:
url = 'https://companium.ru/id/'

headers = {
    'Accept' : '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
    Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36'
    }

In [3]:
def get_dop_okved(ogrn):
    seconds = np.random.randint(10, 16)
    time.sleep(seconds)
    response = requests.get(url + ogrn + '/activity', headers=headers)
    if not response.ok:
        return None
    else: 
        soup = BeautifulSoup(response.text)
        text = soup.find('div', class_='container-xl mb-5').text
        dop_okved_code = re.findall(r'\d{2}[.]\d{1,2}[.]?\d{0,2}[.]?\d{0,2}', text)[1:]
        dop_okved_decoding = [name.replace('\n', '') for name in re.split(r'\d{2}[.]\d{1,2}[.]?\d{0,2}[.]?\d{0,2}', text)[2:]]
    return dop_okved_code, dop_okved_decoding

In [53]:
data_01 = pd.read_excel('01_код_407_записей.xlsx', skiprows=2)
data_01.insert(loc= 7, column='Дополнительный вид деятельности', value=np.array([np.nan]*len(data_01)).reshape(-1, 1))
data_01.insert(loc= 8, column='Дополнительный вид деятельности (расшифровка)', value=np.array([np.nan]*len(data_01)).reshape(-1, 1))
data_01.head()

,№ п/п,Наименование / ФИО,Тип субъекта,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Регион,...,Дата исключения из реестра,Телефон,E-mail,WWW,Наличие лицензий,"Наличие заключенных договоров, контрактов","Производство инновационной, высокотехнологичной продукции",Участие в программах партнерства,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год
0,1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""",Юридическое лицо,Среднее предприятие,1112361001103,2361007110,01.11.1 Выращивание зерновых культур,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,95.0
1,2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОНОВА""",Юридическое лицо,Малое предприятие,1167847206450,7838053444,01.50 Смешанное сельское хозяйство,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,93.0
2,3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОПРОМЫШЛЕННАЯ КОМПАНИ...",Юридическое лицо,Малое предприятие,1022300768555,2304036179,01.2 Выращивание многолетних культур,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,56.0
3,4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОФИРМА КУХАРИВСКАЯ""",Юридическое лицо,Среднее предприятие,1022303857003,2331012177,01.41.11 Разведение молочного крупного рогатог...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,165.0
4,5,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БЕЛАГРО""",Юридическое лицо,Среднее предприятие,1042301302713,2303022021,01.50 Смешанное сельское хозяйство,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,112.0


In [68]:
data_01 = pd.read_excel('01_код_с_доп_оквэдами.xlsx')
idx_nan = np.where(np.isnan(data_01['Дополнительный вид деятельности']))[0]

for idx in tqdm(idx_nan):
    ogrn = str(data_01['ОГРН'][idx])
    result = get_dop_okved(ogrn)
    if result != None:
        data_01['Дополнительный вид деятельности'][idx] = result[0]
        data_01['Дополнительный вид деятельности (расшифровка)'][idx] = result[1]
    else:
        data_01.to_excel('01_код_с_доп_оквэдами.xlsx')
        break

100%|██████████████████████████████████████████████████████████████████████████████| 407/407 [1:27:22<00:00, 12.88s/it]


In [72]:
data_01.to_excel('01_код_с_доп_оквэдами.xlsx')

In [5]:
data_10 = pd.read_excel('10_код_208_записей.xlsx', skiprows=2)
data_10.insert(loc= 7, column='Дополнительный вид деятельности', value=np.array([np.nan]*len(data_10)).reshape(-1, 1))
data_10.insert(loc= 8, column='Дополнительный вид деятельности (расшифровка)', value=np.array([np.nan]*len(data_10)).reshape(-1, 1))
data_10.head()

,№ п/п,Наименование / ФИО,Тип субъекта,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Регион,...,Дата исключения из реестра,Телефон,E-mail,WWW,Наличие лицензий,"Наличие заключенных договоров, контрактов","Производство инновационной, высокотехнологичной продукции",Участие в программах партнерства,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год
0,1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АРМАВИРСКИЙ ХЛЕБОПРОДУКТ""",Юридическое лицо,Среднее предприятие,1022300640482,2302010672,10.61.2 Производство муки из зерновых культур,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,134.0
1,2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЕЙСКХЛЕБ""",Юридическое лицо,Среднее предприятие,1022301121677,2306009371,10.71 Производство хлеба и мучных кондитерских...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,143.0
2,3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОМБИКОРМОВЫЙ ЗАВОД ""ПАВ...",Юридическое лицо,Малое предприятие,1022304478503,2346008141,10.91 Производство готовых кормов для животных...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,91.0
3,4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОМБИНАТ ""ДАРЫ КУБАНИ""",Юридическое лицо,Малое предприятие,1022301807330,2311067604,10.84 Производство приправ и пряностей,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,34.0
4,5,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЛАБИНСКИЙ ХЛЕБОЗАВОД""",Юридическое лицо,Среднее предприятие,1022302352313,2314005765,10.71 Производство хлеба и мучных кондитерских...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,89.0


In [7]:
data_10 = pd.read_excel('10_код_с_доп_оквэдами.xlsx')
idx_nan = np.where(np.isnan(data_10['Дополнительный вид деятельности']))[0]

for idx in tqdm(idx_nan):
    ogrn = str(data_10['ОГРН'][idx])
    result = get_dop_okved(ogrn)
    if result != None:
        data_10['Дополнительный вид деятельности'][idx] = result[0]
        data_10['Дополнительный вид деятельности (расшифровка)'][idx] = result[1]
    else:
        data_10.to_excel('10_код_с_доп_оквэдами.xlsx')
        break

100%|████████████████████████████████████████████████████████████████████████████████| 208/208 [44:39<00:00, 12.88s/it]


In [73]:
data_10.to_excel('10_код_с_доп_оквэдами.xlsx')

In [5]:
data_41 = pd.read_excel('41_код_329_записей.xlsx', skiprows=2)
data_41.insert(loc= 7, column='Дополнительный вид деятельности', value=np.array([np.nan]*len(data_41)).reshape(-1, 1))
data_41.insert(loc= 8, column='Дополнительный вид деятельности (расшифровка)', value=np.array([np.nan]*len(data_41)).reshape(-1, 1))
data_41.head()

,№ п/п,Наименование / ФИО,Тип субъекта,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Регион,...,Дата исключения из реестра,Телефон,E-mail,WWW,Наличие лицензий,"Наличие заключенных договоров, контрактов","Производство инновационной, высокотехнологичной продукции",Участие в программах партнерства,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год
0,1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БИЛДИНГ""",Юридическое лицо,Малое предприятие,1062310006351,2310111297,41.20 Строительство жилых и нежилых зданий,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,15.0
1,2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КРАСНОДАРСКИЙ ЗАВОД ЖЕЛЕ...",Юридическое лицо,Среднее предприятие,1022301812785,2311011320,41.20 Строительство жилых и нежилых зданий,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,11.0
2,3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КУРОРТСТРОЙ""",Юридическое лицо,Малое предприятие,1022300768270,2304016990,41.20 Строительство жилых и нежилых зданий,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,71.0
3,4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТР...",Юридическое лицо,Малое предприятие,1022301116144,2306000450,41.20 Строительство жилых и нежилых зданий,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,50.0
4,5,АКЦИОНЕРНОЕ ОБЩЕСТВО МЕЖРЕГИОНАЛЬНАЯ ПЕРЕДВИЖН...,Юридическое лицо,Среднее предприятие,1022301428050,2309001518,41.20 Строительство жилых и нежилых зданий,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,44.0


In [7]:
data_41 = pd.read_excel('41_код_с_доп_оквэдами.xlsx')
idx_nan = np.where(np.isnan(data_41['Дополнительный вид деятельности']))[0]

for idx in tqdm(idx_nan):
    ogrn = str(data_41['ОГРН'][idx])
    result = get_dop_okved(ogrn)
    if result != None:
        data_41['Дополнительный вид деятельности'][idx] = result[0]
        data_41['Дополнительный вид деятельности (расшифровка)'][idx] = result[1]
    else:
        data_41.to_excel('41_код_с_доп_оквэдами.xlsx')
        break

100%|██████████████████████████████████████████████████████████████████████████████| 329/329 [1:10:00<00:00, 12.77s/it]


In [77]:
data_41.to_excel('41_код_с_доп_оквэдами.xlsx')

In [4]:
data_56 = pd.read_excel('56_код_279_записей.xlsx', skiprows=2)
data_56.insert(loc= 7, column='Дополнительный вид деятельности', value=np.array([np.nan]*len(data_56)).reshape(-1, 1))
data_56.insert(loc= 8, column='Дополнительный вид деятельности (расшифровка)', value=np.array([np.nan]*len(data_56)).reshape(-1, 1))
data_56.head()

,№ п/п,Наименование / ФИО,Тип субъекта,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Регион,...,Дата исключения из реестра,Телефон,E-mail,WWW,Наличие лицензий,"Наличие заключенных договоров, контрактов","Производство инновационной, высокотехнологичной продукции",Участие в программах партнерства,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год
0,1,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""АЛГА""",Юридическое лицо,Малое предприятие,1022300772780,2304038296,56.10.1 Деятельность ресторанов и кафе с полны...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,33.0
1,2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АТОН""",Юридическое лицо,Малое предприятие,1132365001405,2365022053,56.10 Деятельность ресторанов и услуги по дост...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,11.0
2,3,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕС...",Юридическое лицо,Малое предприятие,1162367053980,2319061523,56.10 Деятельность ресторанов и услуги по дост...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,12.0
3,4,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕС...",Юридическое лицо,Малое предприятие,1082304000613,2304052043,56.10 Деятельность ресторанов и услуги по дост...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,29.0
4,5,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ШКО...",Юридическое лицо,Малое предприятие,1152370000860,2370004878,56.29 Деятельность предприятий общественного п...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,73.0


In [81]:
data_56 = pd.read_excel('56_код_с_доп_оквэдами.xlsx')
idx_nan = np.where(np.isnan(data_56['Дополнительный вид деятельности']))[0]

for idx in tqdm(idx_nan):
    ogrn = str(data_56['ОГРН'][idx])
    result = get_dop_okved(ogrn)
    if result != None:
        data_56['Дополнительный вид деятельности'][idx] = result[0]
        data_56['Дополнительный вид деятельности (расшифровка)'][idx] = result[1]
    else:
        data_56.to_excel('56_код_с_доп_оквэдами.xlsx')
        break

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [14:40<00:00, 16.01s/it]


In [78]:
data_56.to_excel('56_код_с_доп_оквэдами.xlsx')

In [83]:
data_86 = pd.read_excel('86_код_262_записей.xlsx', skiprows=2)
data_86.insert(loc= 7, column='Дополнительный вид деятельности', value=np.array([np.nan]*len(data_86)).reshape(-1, 1))
data_86.insert(loc= 8, column='Дополнительный вид деятельности (расшифровка)', value=np.array([np.nan]*len(data_86)).reshape(-1, 1))
data_86.head()

,№ п/п,Наименование / ФИО,Тип субъекта,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Регион,...,Дата исключения из реестра,Телефон,E-mail,WWW,Наличие лицензий,"Наличие заключенных договоров, контрактов","Производство инновационной, высокотехнологичной продукции",Участие в программах партнерства,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год
0,1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОЛНА РЕЗОРТ ЭНД СПА""",Юридическое лицо,Среднее предприятие,1022302715214,2317011051,86.90.4 Деятельность санаторно-курортных орган...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,21.0
1,2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ПАНСИОНАТ ""ШЕПСИ""",Юридическое лицо,Малое предприятие,1022304915742,2355005066,86.90.4 Деятельность санаторно-курортных орган...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,39.0
2,3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ПАНСИОНАТ С ЛЕЧЕНИЕМ ""ИМ...",Юридическое лицо,Малое предприятие,1022304916590,2355015040,86.90.4 Деятельность санаторно-курортных орган...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,42.0
3,4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""САНАТОРИЙ ""АВРОРА""",Юридическое лицо,Малое предприятие,1022302787737,2318020299,86.90.4 Деятельность санаторно-курортных орган...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,66.0
4,5,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""САНАТОРИЙ ""БЕЛЫЕ НОЧИ""",Юридическое лицо,Среднее предприятие,1022302786880,2318024818,86.90.4 Деятельность санаторно-курортных орган...,NaN,NaN,23 - Краснодарский край,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,Нет,Нет,133.0


In [85]:
data_86 = pd.read_excel('86_код_с_доп_оквэдами.xlsx')
idx_nan = np.where(np.isnan(data_86['Дополнительный вид деятельности']))[0]

for idx in tqdm(idx_nan):
    ogrn = str(data_86['ОГРН'][idx])
    result = get_dop_okved(ogrn)
    if result != None:
        data_86['Дополнительный вид деятельности'][idx] = result[0]
        data_86['Дополнительный вид деятельности (расшифровка)'][idx] = result[1]
    else:
        data_86.to_excel('86_код_с_доп_оквэдами.xlsx')
        break

100%|██████████████████████████████████████████████████████████████████████████████| 262/262 [1:12:47<00:00, 16.67s/it]


In [88]:
data_86.to_excel('86_код_с_доп_оквэдами.xlsx')